# Web Server Log Analysis - Python Take-Home Assessment

## Overview
This assessment involves analyzing the Calgary HTTP dataset, which contains approximately one year's worth of HTTP requests to the University of Calgary's Computer Science web server. You'll work with real-world web server log data to extract meaningful insights and demonstrate your Python data analysis skills.

## Part 1: Data Loading and Cleaning

### Instructions

* Work in the cells below - You can add as many cells as needed for data loading, cleaning, and exploration
* Import required libraries
* Implement data loading and cleaning - Create functions to download, parse, and clean the log data
* Explore the data - Understand the structure and identify any data quality issues

In [1]:
# You can write your code here for data loading, cleaning, and exploration. Add cells as necessary.

In [1]:
# Import required libaries

import pandas as pd
import apache_log_parser

In [3]:
# Create the log_format

log_format = '%h %l %u %t "%r" %>s %b' 

# Create a log parser

parser = apache_log_parser.make_parser(log_format)

# Creating a empty list to insert data

log_data = []

# Log parsing process

with open('calgary_access_log', 'r', encoding='utf-8', errors='replace') as file:
    for line in file:
        try:
            parsed_log = parser(line)
            log_data.append(parsed_log)
        except Exception as e:
            
            continue
df = pd.DataFrame(log_data)

In [7]:
df.head()

,remote_host,remote_logname,remote_user,time_received,time_received_datetimeobj,time_received_isoformat,time_received_tz_datetimeobj,time_received_tz_isoformat,time_received_utc_datetimeobj,time_received_utc_isoformat,...,request_url_fragment,request_url_username,request_url_password,request_url_hostname,request_url_port,request_url_query_dict,request_url_query_list,request_url_query_simple_dict,status,response_bytes_clf
0,local,-,-,[24/Oct/1994:13:41:41 -0600],1994-10-24 13:41:41,1994-10-24T13:41:41,1994-10-24 13:41:41-06:00,1994-10-24T13:41:41-06:00,1994-10-24 19:41:41+00:00,1994-10-24T19:41:41+00:00,...,,None,None,None,None,{},[],{},200,150
1,local,-,-,[24/Oct/1994:13:41:41 -0600],1994-10-24 13:41:41,1994-10-24T13:41:41,1994-10-24 13:41:41-06:00,1994-10-24T13:41:41-06:00,1994-10-24 19:41:41+00:00,1994-10-24T19:41:41+00:00,...,,None,None,None,None,{},[],{},200,1210
2,local,-,-,[24/Oct/1994:13:43:13 -0600],1994-10-24 13:43:13,1994-10-24T13:43:13,1994-10-24 13:43:13-06:00,1994-10-24T13:43:13-06:00,1994-10-24 19:43:13+00:00,1994-10-24T19:43:13+00:00,...,,None,None,None,None,{},[],{},200,3185
3,local,-,-,[24/Oct/1994:13:43:14 -0600],1994-10-24 13:43:14,1994-10-24T13:43:14,1994-10-24 13:43:14-06:00,1994-10-24T13:43:14-06:00,1994-10-24 19:43:14+00:00,1994-10-24T19:43:14+00:00,...,,None,None,None,None,{},[],{},200,2555
4,local,-,-,[24/Oct/1994:13:43:15 -0600],1994-10-24 13:43:15,1994-10-24T13:43:15,1994-10-24 13:43:15-06:00,1994-10-24T13:43:15-06:00,1994-10-24 19:43:15+00:00,1994-10-24T19:43:15+00:00,...,,None,None,None,None,{},[],{},200,36403


# Data Cleaning
* Parse timestamp strings into `datetime` objects
* Extract file extensions from the filename field
* Handle missing or malformed data entries
* Convert data types appropriately (integers, strings, etc.)
* Remove or flag invalid log entries

In [181]:
# Converting string to date format

df['timestamp'] = pd.to_datetime(df['time_received_datetimeobj'])

In [13]:
# Extract extension from request url
import os
df['extension'] = df['request_url'].apply(lambda x: os.path.splitext(x)[1])

In [9]:
# Change datatype

In [19]:
df['status'] = pd.to_numeric(df['status'], errors='coerce')
df['response_bytes_clf'] = pd.to_numeric(df['response_bytes_clf'], errors='coerce')
df['remote_host'] = df['remote_host'].astype('category')
df['remote_user'] = df['remote_user'].astype('category')
df['extension'] = df['extension'].astype(str)
df['request_url'] = df['request_url'].astype(str)

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724910 entries, 0 to 724909
Data columns (total 30 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   remote_host                    724910 non-null  category      
 1   remote_logname                 724910 non-null  object        
 2   remote_user                    724910 non-null  category      
 3   time_received                  724910 non-null  object        
 4   time_received_datetimeobj      724910 non-null  datetime64[ns]
 5   time_received_isoformat        724910 non-null  object        
 6   time_received_tz_datetimeobj   724910 non-null  object        
 7   time_received_tz_isoformat     724910 non-null  object        
 8   time_received_utc_datetimeobj  724910 non-null  object        
 9   time_received_utc_isoformat    724910 non-null  object        
 10  request_first_line             724910 non-null  object        
 11  

In [23]:
# Remove invaild entries
df_clean = df.dropna(subset=['time_received_datetimeobj', 'status', 'response_bytes_clf', 'request_url', 'extension'])

## ⚠️ IMPORTANT: Template Questions Section
**DO NOT MODIFY THE TEMPLATE BELOW THIS POINT**

The following section contains the assessment questions. You may add cells above this section for data loading, cleaning, and exploration, but do not modify the function signatures or structure of the questions below.

## Part 2: Analysis Questions

### Instructions

* Implement each function according to its docstring specifications
* Use the cleaned data you prepared in Part 1
* Ensure your functions return the exact data types specified
* Test your functions to verify they work correctly
* You may add helper functions, but keep the main function signatures unchanged

### Q1: Count of total log records

In [185]:
def total_log_records(df) -> int:
    """
    Q1: Count of total log records.

    Objective:
        Determine the total number of HTTP log entries in the dataset.
        Each line in the log file represents one HTTP request.

    Returns:
        int: Total number of log entries.
    """

    # TODO: Implement logic to count log records

    return len(df)  # Placeholder return


answer1 = total_log_records(df_clean)
print("Answer :")
print(answer1)

Answer :
666804


### Q2: Count of unique hosts

In [101]:
def unique_host_count(df) -> int:
    """
    Q2: Count of unique hosts.

    Objective:
        Determine how many distinct hosts accessed the server.

    Returns:
        int: Number of unique hosts.
    """

    # TODO: Implement logic to count unique hosts

    return df['remote_host'].nunique()  # Placeholder return


answer2 = unique_host_count(df_clean)
print("Answer 2:")
print(answer2)

Answer 2:
2


### Q3: Date-wise unique filename counts

In [25]:
def datewise_unique_filename_counts(df) -> dict[str, int]:
    """
    Q3: Date-wise unique filename counts.

    Objective:
        For each date, count the number of unique filenames that accessed the server.
        The date should be in 'dd-MMM-yyyy' format (e.g., '01-Jul-1995').

    Returns:
        dict: A dictionary mapping each date to its count of unique filenames.
              Example: {'01-Jul-1995': 123, '02-Jul-1995': 150}
    """

    # TODO: Implement logic for date-wise unique filename counts
    df['datetime'] = pd.to_datetime(df['time_received_datetimeobj'], errors='coerce')
    df = df.dropna(subset=['datetime', 'request_url'])
    df['date_str'] = df['datetime'].dt.strftime('%d-%b-%Y')
    result = df.groupby('date_str')['request_url'].nunique().to_dict()

    return result  # Placeholder return


answer3 = datewise_unique_filename_counts(df_clean)
print("Answer 3:")
print(answer3)

C:\Users\user\AppData\Local\Temp\ipykernel_4744\2167715360.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['time_received_datetimeobj'], errors='coerce')


Answer 3:
{'01-Apr-1995': 408, '01-Aug-1995': 621, '01-Dec-1994': 261, '01-Feb-1995': 582, '01-Jan-1995': 86, '01-Jul-1995': 297, '01-Jun-1995': 572, '01-Mar-1995': 556, '01-May-1995': 454, '01-Nov-1994': 392, '01-Oct-1995': 526, '01-Sep-1995': 320, '02-Apr-1995': 435, '02-Aug-1995': 765, '02-Dec-1994': 312, '02-Feb-1995': 502, '02-Jan-1995': 138, '02-Jul-1995': 345, '02-Jun-1995': 495, '02-Mar-1995': 576, '02-May-1995': 681, '02-Nov-1994': 410, '02-Oct-1995': 828, '02-Sep-1995': 351, '03-Apr-1995': 746, '03-Aug-1995': 558, '03-Dec-1994': 172, '03-Feb-1995': 539, '03-Jan-1995': 297, '03-Jul-1995': 424, '03-Jun-1995': 381, '03-Mar-1995': 499, '03-May-1995': 597, '03-Nov-1994': 431, '03-Oct-1995': 823, '03-Sep-1995': 207, '04-Apr-1995': 773, '04-Aug-1995': 667, '04-Dec-1994': 201, '04-Feb-1995': 542, '04-Jan-1995': 326, '04-Jul-1995': 588, '04-Jun-1995': 345, '04-Mar-1995': 391, '04-May-1995': 659, '04-Nov-1994': 384, '04-Oct-1995': 877, '04-Sep-1995': 334, '05-Apr-1995': 844, '05-Aug-19

### Q4: Number of 404 response codes

In [130]:
def count_404_errors(df) -> int:
    """
    Q4: Number of 404 response codes.

    Objective:
        Count how many times the HTTP 404 Not Found status appears in the logs.

    Returns:
        int: Number of 404 errors.
    """

    # TODO: Implement logic to count 404 errors
    

    return (df['status'] == 404).sum()  # Placeholder return


answer4 = count_404_errors(df)
print("Answer 4:")

print(answer4)

Answer 4:
23586


### Q5: Top 15 filenames with 404 responses

In [239]:
def top_15_filenames_with_404(df : str, n : int) -> list[tuple[str, int]]:
    """
    Q5: Top 15 filenames with 404 responses.

    Objective:
        Identify which requested URLs most frequently resulted in a 404 error.
        Return the top 15 filenames sorted by frequency.

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
              Example: [('index.html', 200), ...]
    """

    # TODO: Implement logic to find top 15 filenames with 404
    filename_counts = (df[df['status'] == 404]['request_url']).value_counts().head(n)

    return list(filename_counts.items())  # Placeholder return


answer5 = top_15_filenames_with_404(df, 15)
print("Answer 5:")
print(answer5)


Answer 5:
[('index.html', 4691), ('4115.html', 900), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 257), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('488.gif', 148), ('3414.gif', 148), ('40.html', 148), ('9678.gif', 142)]


### Q6: Top 15 file extension with 404 responses

In [235]:
def top_15_ext_with_404(df, n : int) -> list[tuple[str, int]]:
    """
    Q6: Top 15 file extensions with 404 responses.

    Objective:
        Find which file extensions generated the most 404 errors.
        Return the top 15 sorted by number of 404s.

    Returns:
        list: A list of tuples (extension, count), sorted by count in descending order.
              Example: [('html', 45), ...]
    """

    # TODO: Implement logic to find top 15 extensions with 404
    top_n_ext = df[df['status'] == 404]['extension'].value_counts().head(n)
    return list(top_n_ext.items())  # Placeholder return


answer6 = top_15_ext_with_404(df, 15)
print("Answer 6:")
print(answer6)

Answer 6:
[('.html', 12166), ('.gif', 7210), ('.xbm', 826), ('.ps', 757), ('.jpg', 520), ('.txt', 497), ('', 355), ('.GIF', 135), ('.htm', 108), ('.cgi', 77), ('.com', 46), ('.gif"', 45), ('.Z', 41), ('.dvi', 40), ('.ca', 37)]


### Q7: Total bandwidth transferred per day for the month of July 1995

In [206]:
def total_bandwidth_per_day(df) -> dict[str, int]:
    """
    Q7: Total bandwidth transferred per day for the month of July 1995.

    Objective:
        Sum the number of bytes transferred per day.
        Skip entries where the byte field is missing or '-'.

    Returns:
        dict: A dictionary mapping each date to total bytes transferred.
              Example: {'01-Jul-1995': 123456789, ...}
    """

    # TODO: Implement logic to compute total bandwidth per day
    df.dropna(subset= ['response_bytes_clf', 'time_received_datetimeobj'])

    df_july = df[df['time_received_datetimeobj'].dt.strftime('%b %Y') == 'Jul 1995']

    df_july['date_str'] = df_july['time_received_datetimeobj'].dt.strftime('%d %b %Y')
    bandwidth_by_date = df_july.groupby('date_str')['response_bytes_clf'].sum().astype(int)

    return bandwidth_by_date.to_dict()  # Placeholder return


answer7 = total_bandwidth_per_day(df_clean)
print("Answer 7:")
print(answer7)

Answer 7:
{'01 Jul 1995': 11349799, '02 Jul 1995': 8656918, '03 Jul 1995': 13596612, '04 Jul 1995': 26573988, '05 Jul 1995': 19541225, '06 Jul 1995': 19755015, '07 Jul 1995': 9427822, '08 Jul 1995': 5403491, '09 Jul 1995': 4660556, '10 Jul 1995': 14917754, '11 Jul 1995': 22507207, '12 Jul 1995': 17367065, '13 Jul 1995': 15989234, '14 Jul 1995': 19186430, '15 Jul 1995': 15773233, '16 Jul 1995': 9016378, '17 Jul 1995': 19601338, '18 Jul 1995': 17099761, '19 Jul 1995': 17851725, '20 Jul 1995': 20752623, '21 Jul 1995': 25491617, '22 Jul 1995': 8136259, '23 Jul 1995': 9593870, '24 Jul 1995': 22308265, '25 Jul 1995': 24561635, '26 Jul 1995': 24995540, '27 Jul 1995': 25969995, '28 Jul 1995': 36460693, '29 Jul 1995': 11700624, '30 Jul 1995': 23189598, '31 Jul 1995': 30730715}


C:\Users\user\AppData\Local\Temp\ipykernel_14832\3596242896.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_july['date_str'] = df_july['time_received_datetimeobj'].dt.strftime('%d %b %Y')


### Q8: Hourly request distribution

In [216]:
def hourly_request_distribution(df) -> dict[int, int]:
    """
    Q8: Hourly request distribution.

    Objective:
        Count the number of requests made during each hour (00 to 23).
        Useful for understanding traffic peaks.

    Returns:
        dict: A dictionary mapping hour (int) to request count.
              Example: {0: 120, 1: 90, ..., 23: 80}
    """

    # TODO: Implement logic for hourly distribution
    df.dropna(subset = ['time_received_datetimeobj'])
    hourly_req = df['time_received_datetimeobj'].dt.hour.value_counts().sort_index()

    return hourly_req.to_dict()  # Placeholder return


answer8 = hourly_request_distribution(df_clean)
print("Answer 8:")
print(answer8)

Answer 8:
{0: 17206, 1: 13196, 2: 11733, 3: 10027, 4: 9369, 5: 10143, 6: 12280, 7: 15550, 8: 24664, 9: 31115, 10: 39713, 11: 43766, 12: 43085, 13: 47338, 14: 50087, 15: 46337, 16: 46849, 17: 41378, 18: 30435, 19: 28219, 20: 27169, 21: 25109, 22: 21873, 23: 20163}


### Q9: Top 10 most requested filenames

In [233]:
def top_10_most_requested_filenames(df, n) -> list[tuple[str, int]]:
    """
    Q9: Top 10 most requested filenames.

    Objective:
        Identify the most commonly requested URLs (irrespective of status code).

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
                Example: [('index.html', 500), ...]
    """

    # TODO: Implement logic to find top 10 most requested filenames

    top_n_filename = df['request_url'].value_counts().head(n)

    return list(top_n_filename.items())  # Placeholder return


answer9 = top_10_most_requested_filenames(df_clean, 10)
print("Answer 9:")
print(answer9)

Answer 9:
[('index.html', 104045), ('3.gif', 24005), ('2.gif', 23595), ('4.gif', 8017), ('244.gif', 5144), ('5.html', 5004), ('4097.gif', 4865), ('8870.jpg', 4488), ('6733.gif', 4278), ('8472.gif', 3843)]


### Q10: HTTP response code distribution

In [255]:
def response_code_distribution(df) -> dict[int, int]:
    """
    Q10: HTTP response code distribution.

    Objective:
        Count how often each HTTP status code appears in the logs.

    Returns:
        dict: A dictionary mapping HTTP status codes (as int) to their frequency.
              Example: {200: 150000, 404: 3000}
    """

    # TODO: Implement logic for response code counts
    df = df.dropna(subset=['status'])
    count_status = df['status'].value_counts().sort_index()

    return count_status.to_dict()  # Placeholder return


answer10 = response_code_distribution(df_clean)
print("Answer 10:")
print(answer10)

Answer 10:
{200: 568348, 302: 640, 304: 97792, 403: 2, 500: 22}


In [259]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace analysis.ipynb

[NbConvertApp] Converting notebook analysis.ipynb to notebook
[NbConvertApp] Writing 18989 bytes to analysis.ipynb
